# Covered Calls Leap Trader

## Imports

In [166]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.io as pio
import yfinance as yf

from dateutil.relativedelta import relativedelta

from tqdm import tqdm
tqdm.pandas()

## Params

In [167]:
# Difference between long and short term gains (federal change shown below).
# However state by state laws are different... Feel free to look at 
# Input `TAX_ADVANTAGE` as a percent
TAX_ADVANTAGE = .17

# Input `TAX_RATE` as a percent
TAX_RATE = 0.37 + .1075

# Score risk tolerance by standard deviations (cumulative coverage percentage below)
RISK_TOLERANCE = 1

# How much each option takes to trade (this number is based of Schwab)
FEE = .65

# Get options data `AFTER_DATE`
AFTER_DATE = pd.to_datetime('2000-01-01')
BEFORE_DATE = pd.to_datetime('2025-08-08')

# Sets a maximum days to expiration
MAX_DAYS_TILL_EXPIRATION = 400

# Sets the ticker to trade
TICKERS = ['QQQ', 'SPY']

### This shows the difference between long and short term gains (Last Updated 2024)

#### Single Filers

| Money Made                | Tax Rate Difference (Short-Term - Long-Term) |
|---------------------------|----------------------------------------------|
| $0 – $47,025              | 10% – 0% = 10% or 12% – 0% = 12%            |
| $47,026 – $100,525        | 22% – 15% = 7%                              |
| $100,526 – $191,950       | 24% – 15% = 9%                              |
| $191,951 – $243,725       | 32% – 15% = 17%                             |
| $243,726 – $518,900       | 35% – 15% = 20%                             |
| $518,901 or more          | 37% – 20% = 17%                             |

#### Married Filing Jointly

| Money Made                | Tax Rate Difference (Short-Term - Long-Term) |
|---------------------------|----------------------------------------------|
| $0 – $94,050              | 10% – 0% = 10% or 12% – 0% = 12%            |
| $94,051 – $201,050        | 22% – 15% = 7%                              |
| $201,051 – $383,900       | 24% – 15% = 9%                              |
| $383,901 – $487,450       | 32% – 15% = 17%                             |
| $487,451 – $583,750       | 35% – 15% = 20%                             |
| $583,751 or more          | 37% – 20% = 17%                             |


### This Shows The Percent Chance That A Return Happens Outside Said Standard Devitation Range
| **n (Number of Standard Deviations)** | **Cumulative Coverage (%)**    |
|----------------------------------------|----------------------------------|
| 1σ                                     | 68.27%                          |
| 2σ                                     | 95.45%                          |
| 3σ                                     | 99.73%                          |
| 4σ                                     | 99.9937%                        |
| 5σ                                     | 99.99994%                       |
| 6σ                                     | 99.9999998%                     |
| 7σ                                     | 99.9999999997%                  |
| 8σ                                     | 99.99999999999996%              |
| 9σ                                     | ~100%                           |
| 10σ                                    | ~100%                           |


## Read In File

In [197]:
def pick_closest(group):
    out = []
    for num_weeks in [1, 4, 12]:
        target = group.name + relativedelta(weeks=num_weeks)
        # find the expiration_date closest to target
        closest_exp = group.loc[
            (group['expiration_date'] - target).abs().idxmin(),
            'expiration_date'
        ]
        # grab all rows with that expiration_date
        sub = group[group['expiration_date'] == closest_exp].copy()
        sub['expiration_group'] = num_weeks
        out.append(sub)
    # concatenate all horizons into one DataFrame
    return pd.concat(out, ignore_index=True)


In [ ]:
# Initialize a dictionary to hold dataframes for each ticker
all_options_data = {}
portfolio_option_data = {}
portfolio_stock_metric = {}
portfolio_stock_data = {}

# Iterate through tickers and set dictionary to hold proper data
for TICKER in TICKERS:
    
    print(TICKER)
    
    # Read data
    all_options_data[TICKER] = pd.read_parquet(f'../read_data/data/clean/{TICKER}.parquet')
    portfolio_stock_data[TICKER] = all_options_data[TICKER].groupby('date')['stock_price'].first().sort_index().reset_index()

    # Stock data
    portfolio_stock_metric[TICKER] = portfolio_stock_data[TICKER].copy()
    portfolio_stock_metric[TICKER]['RSI_14']  = ta.rsi(
        portfolio_stock_metric[TICKER]['stock_price'],
        length=14
    )

    # Simple & Exponential Moving Averages (20-day)
    portfolio_stock_metric[TICKER]['SMA_20'] = ta.sma(
        portfolio_stock_metric[TICKER]['stock_price'],
        length=20
    )
    portfolio_stock_metric[TICKER]['EMA_20'] = ta.ema(
        portfolio_stock_metric[TICKER]['stock_price'],
        length=20
    )

    # MACD (12,26,9): line, signal, histogram
    macd = ta.macd(
        portfolio_stock_metric[TICKER]['stock_price'],
        fast=12, slow=26, signal=9
    )
    portfolio_stock_metric[TICKER]['MACD'] = macd['MACD_12_26_9']
    portfolio_stock_metric[TICKER]['MACD_signal'] = macd['MACDs_12_26_9']
    portfolio_stock_metric[TICKER]['MACD_hist'] = macd['MACDh_12_26_9']

    portfolio_stock_metric[TICKER]['high_1w'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=5).max()
    portfolio_stock_metric[TICKER]['low_1w']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=5).min()

    portfolio_stock_metric[TICKER]['high_1m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=21).max()
    portfolio_stock_metric[TICKER]['low_1m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=21).min()

    portfolio_stock_metric[TICKER]['high_3m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=63).max()
    portfolio_stock_metric[TICKER]['low_3m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=63).min()

    portfolio_stock_metric[TICKER]['high_6m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=126).max()
    portfolio_stock_metric[TICKER]['low_6m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=126).min()

    portfolio_stock_metric[TICKER]['high_9m'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=189).max()
    portfolio_stock_metric[TICKER]['low_9m']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=189).min()

    portfolio_stock_metric[TICKER]['high_1y'] = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=252).max()
    portfolio_stock_metric[TICKER]['low_1y']  = portfolio_stock_metric[TICKER]['stock_price'].rolling(window=252).min()
    portfolio_stock_metric[TICKER] = portfolio_stock_metric[TICKER].dropna()
    portfolio_stock_metric[TICKER] = portfolio_stock_metric[TICKER].drop(columns=['stock_price'])

    portfolio_option_data[TICKER] = all_options_data[TICKER].loc[all_options_data[TICKER]['date'].isin(portfolio_stock_metric[TICKER]['date'])]

    portfolio_option_data[TICKER] = all_options_data[TICKER].loc[all_options_data[TICKER]['days_till_expiration'] < MAX_DAYS_TILL_EXPIRATION].groupby('date').progress_apply(pick_closest)
    portfolio_option_data[TICKER] = all_options_data[TICKER].reset_index(drop=True)



QQQ


100%|██████████| 2766/2766 [00:04<00:00, 575.00it/s]


SPY


100%|██████████| 3249/3249 [00:06<00:00, 540.48it/s]


In [199]:
portfolio_option_data[TICKER].groupby('date').count()

,time,stock_price,expiration_date,days_till_expiration,c_delta,c_gamma,c_vega,c_theta,c_rho,c_iv,...,p_gamma,p_vega,p_theta,p_rho,p_iv,p_volume,strike_distance,call_price,put_price,expiration_group
date,,,,,,,,,,,,,,,,,,,,,
2011-01-14,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,297,197,327,327,327,327
2011-01-18,327,327,327,327,327,327,327,327,327,324,...,327,327,327,327,300,183,327,327,327,327
2011-01-26,257,257,257,257,257,257,257,257,257,257,...,257,257,257,257,209,131,257,257,257,257
2011-01-27,258,258,258,258,258,258,258,258,258,202,...,258,258,258,258,258,132,258,258,258,258
2011-01-28,258,258,258,258,258,258,258,258,258,258,...,258,258,258,258,163,109,258,258,258,258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,594,594,594,594,594,594,594,594,594,593,...,594,594,594,594,587,565,594,594,594,594
2023-12-26,377,377,377,377,377,377,377,377,377,377,...,377,377,377,377,373,320,377,377,377,377
2023-12-27,376,376,376,376,376,376,376,376,376,376,...,376,376,376,376,376,311,376,376,376,376


In [200]:
portfolio_calls = {}
portfolio_puts = {}

for TICKER in TICKERS:

    print(TICKER)

    portfolio_calls[TICKER] = portfolio_option_data[TICKER].loc[
        portfolio_option_data[TICKER]['stock_price'] > portfolio_option_data[TICKER]['strike']
    ].groupby(['date', 'expiration_group'], group_keys=False).progress_apply(lambda grp: grp.nsmallest(3, 'strike_distance'))[
        ['date', 'strike', 'days_till_expiration', 'expiration_date', 'stock_price', 'strike_distance', 'call_price', 'expiration_group'] +
        [col for col in portfolio_option_data[TICKER].columns if col.startswith('c_')]
    ].rename(
        columns=lambda col: (
            col[2:]
            if col.startswith('c_') 
            else ('price' if col.endswith('_price') else col)
        )
    ).drop(columns=['volume', 'last', 'size', 'bid', 'ask'], errors='ignore').merge(
          portfolio_stock_metric[TICKER],
          on='date',
          how='left'
    )

    portfolio_puts[TICKER] = portfolio_option_data[TICKER].loc[
        portfolio_option_data[TICKER]['stock_price'] < portfolio_option_data[TICKER]['strike']
    ].groupby(['date', 'expiration_group'], group_keys=False).progress_apply(lambda grp: grp.nsmallest(3, 'strike_distance'))[
        ['date', 'strike', 'days_till_expiration', 'expiration_date', 'stock_price', 'strike_distance', 'put_price', 'expiration_group'] +
        [col for col in portfolio_option_data[TICKER].columns if col.startswith('p_')]
    ].rename(
        columns=lambda col: (
            col[2:]
            if col.startswith('p_') 
            else ('price' if col.endswith('_price') else col)
        )
    ).drop(columns=['volume', 'last', 'size', 'bid', 'ask'], errors='ignore').merge(
          portfolio_stock_metric[TICKER],
          on='date',
          how='left'
    )

QQQ


100%|██████████| 8276/8276 [00:04<00:00, 2036.41it/s]


SPY


100%|██████████| 9747/9747 [00:05<00:00, 1897.21it/s]
